# Motivación:

El cine es una de las formas más influyentes y apreciadas de arte y entretenimiento en todo el mundo. Cada película es una obra maestra única que narra historias cautivadoras, despierta emociones y provoca reflexiones profundas. Detrás de estas obras de arte se encuentra una vasta industria cinematográfica que involucra a miles de profesionales creativos y técnicos. La exploración y comprensión de este mundo es un desafío intrigante, y los datos sobre películas nos brindan una ventana a este universo fascinante.

Este dataset cinematográfico nos sumerge en un viaje a través del tiempo y el espacio cinematográfico. Desde los primeros días del cine mudo hasta las épicas producciones contemporáneas, este conjunto de datos ofrece una visión panorámica de la evolución del cine. Nos permite explorar tendencias en géneros cinematográficos, evaluar la popularidad de las películas a lo largo del tiempo y descubrir cómo factores como el presupuesto y las calificaciones influyen en el éxito de una película.

# Audiencia:

Nuestra audiencia abarca a un amplio espectro de entusiastas del cine, estudiantes, académicos, críticos, profesionales de la industria cinematográfica y cualquier persona interesada en descubrir los secretos detrás de sus películas favoritas. Este conjunto de datos es una herramienta valiosa para varios propósitos:

* Investigación Académica: Investigadores y académicos pueden utilizar estos datos para analizar tendencias en la industria cinematográfica, estudiar la relación entre presupuesto y éxito, o explorar la evolución de géneros a lo largo del tiempo.

* Profesionales de la Industria: Profesionales de la industria cinematográfica pueden obtener información sobre lo que hace que una película sea exitosa y tomar decisiones más informadas en la producción y distribución de películas.

* Críticos de Cine: Críticos y aficionados al cine pueden utilizar los datos para evaluar el impacto de las calificaciones de películas en su popularidad y para descubrir gemas cinematográficas menos conocidas.

* Estudiantes y Curiosos: Los amantes del cine, estudiantes de cine y cualquier persona interesada en la magia del séptimo arte pueden explorar este conjunto de datos para obtener una comprensión más profunda de la historia del cine y las dinámicas detrás de sus películas favoritas.

En resumen, este dataset es una invitación a un emocionante viaje a través del mundo del cine, donde los datos se convierten en historias, y las películas se convierten en datos. Es una herramienta versátil que enriquecerá la experiencia de cualquiera que esté dispuesto a adentrarse en el asombroso universo cinematográfico.

# Preguntas/Hipótesis

## Preguntas Generales:

* ¿Cuál es la evolución a lo largo del tiempo de la cantidad de películas producidas?
* ¿Cuáles son los géneros cinematográficos más populares en diferentes décadas?
* ¿Existe una correlación entre el presupuesto de una película y su éxito en taquilla?
* ¿Cómo influyen las calificaciones de la audiencia en la popularidad de una película?
* ¿Cuál es la distribución de la duración de las películas en el dataset?
* ¿Cuál es la relación entre el voto promedio y la popularidad de una película?

## Preguntas sobre Géneros:

* ¿Cuál es el género cinematográfico más común en el dataset?
* ¿Existen tendencias en la popularidad de géneros específicos a lo largo del tiempo?
* ¿Cuál es la relación entre el presupuesto y el género de una película?
* ¿Cuál es la correlación entre el voto promedio y el género de una película?

## Preguntas sobre Directores y Actores:

* ¿Quiénes son los directores más prolíficos en términos de películas producidas?
* ¿Hay una relación entre la participación de actores/actrices populares y el éxito de una película?
* ¿Cuáles son los actores y actrices más recurrentes en el dataset?

## Preguntas sobre Calificaciones y Éxito:

* ¿Existe una relación entre el voto promedio de una película y su éxito en taquilla?
* ¿Cuál es la tendencia de las calificaciones promedio de las películas a lo largo del tiempo?
* ¿Cómo se distribuyen las calificaciones de las películas entre críticos y audiencias?

## Preguntas sobre Presupuesto y Taquilla:

* ¿Cuál es la relación entre el presupuesto de producción y las ganancias en taquilla?
* ¿Existen géneros cinematográficos que tienden a tener un mayor retorno de inversión (ROI)?
* ¿Cuál es el presupuesto promedio de las películas a lo largo de los años?

## Preguntas sobre Fechas de Lanzamiento:

* ¿Hay meses específicos en los que se lanzan más películas?
* ¿Existe una temporada del año en la que las películas tienden a tener un mejor rendimiento en taquilla?
* ¿Cuál es la tendencia de lanzamiento de películas en diferentes décadas?

Estas son solo algunas de las preguntas e hipótesis que podremos explorar con en nuestro dataset de películas. Dependiendo de los intereses y objetivos, puedemos profundizar en una o más de estas preguntas utilizando técnicas de análisis de datos y visualización. 

# Definición de Objetivo:

El objetivo de este proyecto de data science es proporcionar a la industria cinematográfica y a los interesados en el cine un conjunto de análisis y herramientas que les permitan comprender mejor las tendencias, los factores de éxito y los patrones que influyen en la producción y el rendimiento de las películas. Buscamos extraer conocimientos valiosos de los datos disponibles para tomar decisiones informadas en la producción, distribución y evaluación de películas, así como para anticipar las preferencias de la audiencia.

# Contexto Comercial:

La industria cinematográfica es altamente competitiva y costosa. Las productoras invierten millones de dólares en la creación de películas y buscan maximizar su éxito en taquilla y en plataformas de streaming. Los estudios de cine, las productoras independientes y los inversores buscan constantemente identificar oportunidades de inversión, comprender los factores clave que impulsan el éxito y mitigar riesgos financieros. Además, las plataformas de streaming necesitan conocer las preferencias de su audiencia para ofrecer contenido relevante.

# Problema Comercial:

El problema comercial principal es optimizar la toma de decisiones en la industria cinematográfica. Esto incluye:

* Identificar qué géneros, directores, actores y presupuestos tienden a tener un mejor desempeño en términos de taquilla y calificaciones.
* Evaluar la relación entre la inversión en publicidad y el éxito en taquilla.
* Comprender las preferencias cambiantes de la audiencia a lo largo del tiempo y en diferentes regiones.
* Pronosticar el rendimiento potencial de una película antes de su lanzamiento para tomar decisiones de producción y distribución.
* Descubrir oportunidades de nicho y tendencias emergentes que puedan ser explotadas por productoras independientes y nuevas empresas en la industria cinematográfica.

# Contexto Analítico:

El contexto analítico implica el uso de técnicas de análisis de datos y herramientas de visualización para responder a las preguntas comerciales planteadas. Esto incluye:

* Análisis exploratorio de datos para comprender la distribución y las tendencias en el conjunto de datos de películas.
* Modelado de regresión para evaluar la influencia de diversas variables (presupuesto, género, calificaciones) en el éxito en taquilla.
* Segmentación de la audiencia y detección de patrones de preferencia utilizando algoritmos de aprendizaje automático.
* Creación de modelos predictivos para anticipar el rendimiento de una película en función de sus características y contexto de lanzamiento.
* Desarrollo de cuadros de mando y visualizaciones interactivas para que los interesados puedan explorar los datos y obtener información valiosa.

En resumen, este proyecto de data science busca aprovechar los datos sobre películas para ayudar a la industria cinematográfica a tomar decisiones más acertadas y a satisfacer las necesidades cambiantes de la audiencia, contribuyendo así al éxito continuo del séptimo arte.

# Análisis Exploratorio de Datos (EDA)

# Librerias

In [112]:
# Comencemos importando nuestras librerias
import pandas as pd
import numpy as np
import ast
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

pd.options.display.float_format = '{:.2f}'.format

# Presentación DataSet

In [113]:
url = 'https://raw.githubusercontent.com/ApocalypseScrambler/datascience/main/Movie%20Recommendation%20System/tmdb_5000_movies.csv'
url_2 = 'https://raw.githubusercontent.com/ApocalypseScrambler/datascience/main/Movie%20Recommendation%20System/tmdb_5000_credits.csv'

# Levanto el archivo 
df_movies = pd.read_csv(url,sep = ',')
# Reordenemos el dataframe
df_movies = df_movies[['id', 'budget', 'genres', 'homepage', 'keywords', 'original_language', 'original_title', 'overview', 'popularity', 'production_companies', 'production_countries',
                'release_date', 'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'vote_average', 'vote_count']]
# Importamos el segundo archivo
df_credits = pd.read_csv(url_2,sep = ',')

#Unamos los dos dataframes
df_movies = pd.merge(df_movies, df_credits, on="id", how="outer")

#Cambiemos el indice
df_movies.index = df_movies['id']
df_movies = df_movies.drop('id', axis='columns')

# Vamos algunos de sus datos
print(df_movies.head())


           budget                                             genres  \
id                                                                     
19995   237000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
285     300000000  [{"id": 12, "name": "Adventure"}, {"id": 14, "...   
206647  245000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   
49026   250000000  [{"id": 28, "name": "Action"}, {"id": 80, "nam...   
49529   260000000  [{"id": 28, "name": "Action"}, {"id": 12, "nam...   

                                            homepage  \
id                                                     
19995                    http://www.avatarmovie.com/   
285     http://disney.go.com/disneypictures/pirates/   
206647   http://www.sonypictures.com/movies/spectre/   
49026             http://www.thedarkknightrises.com/   
49529           http://movies.disney.com/john-carter   

                                                 keywords original_language  \
id                     

In [114]:
#Descripción de las columnas
df_movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4803 entries, 19995 to 25975
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   budget                4803 non-null   int64  
 1   genres                4803 non-null   object 
 2   homepage              1712 non-null   object 
 3   keywords              4803 non-null   object 
 4   original_language     4803 non-null   object 
 5   original_title        4803 non-null   object 
 6   overview              4800 non-null   object 
 7   popularity            4803 non-null   float64
 8   production_companies  4803 non-null   object 
 9   production_countries  4803 non-null   object 
 10  release_date          4802 non-null   object 
 11  revenue               4803 non-null   int64  
 12  runtime               4801 non-null   float64
 13  spoken_languages      4803 non-null   object 
 14  status                4803 non-null   object 
 15  tagline              

## Columnas 

* budget - El presupuesto con el que se hizo la película.
* genres - Los géneros de la película, acción, comedia, suspense, etc.
* homepage - Un enlace a la página de inicio de la película.
* keywords - Las palabras clave o etiquetas relacionadas con la película.
* original_language - El idioma en el que se hizo la película.
* original_title - El título de la película antes de la traducción o adaptación.
* overview - Una breve descripción de la película.
* popularity - Una cantidad numérica que especifica la popularidad de la película.
* production_companies - La casa productora de la película.
* production_countries - El país en el que se produjo.
* release_date - La fecha en que fue lanzada.
* revenue - Los ingresos mundiales generados por la película..
* runtime - El tiempo de ejecución de la película en minutos.
* status - "Released" or "Rumored".
* tagline - Eslogan de la película.
* title - Título de la película.
* vote_average - Calificaciones promedio que recibió la película.
* vote_count - el recuento de votos recibidos.
* cast - El nombre de los actores principales y secundarios.
* crew - El nombre del Director, Editor, Compositor, Escritor, etc.

In [115]:
#Como podemos apreciar tenemos dos columnas con los mismos datos luego de la unión. Eliminemos una de ellas.
print(df_movies[['title_x','title_y']].head())

#Eliminemos la columna repetida
df_movies = df_movies.drop(['title_y'], axis=1)

#Cambiamos el nombre
df_movies.rename(columns={'title_x': 'title'}, inplace=True)

                                         title_x  \
id                                                 
19995                                     Avatar   
285     Pirates of the Caribbean: At World's End   
206647                                   Spectre   
49026                      The Dark Knight Rises   
49529                                John Carter   

                                         title_y  
id                                                
19995                                     Avatar  
285     Pirates of the Caribbean: At World's End  
206647                                   Spectre  
49026                      The Dark Knight Rises  
49529                                John Carter  


In [116]:
#Veamos ahora que campos nos quedan en null
print('Nulls')
print('----------------------------')
print(df_movies.isnull().sum())

Nulls
----------------------------
budget                     0
genres                     0
homepage                3091
keywords                   0
original_language          0
original_title             0
overview                   3
popularity                 0
production_companies       0
production_countries       0
release_date               1
revenue                    0
runtime                    2
spoken_languages           0
status                     0
tagline                  844
title                      0
vote_average               0
vote_count                 0
cast                       0
crew                       0
dtype: int64


In [117]:
# Eliminamos los registros con nulos de release_date y overview ya que son pocos
df_movies = df_movies.dropna(subset=['release_date', 'overview'])

# Tambien nos aseguramos que el campo release_date sea considerado como una fecha como tal
df_movies['release_date'] = pd.to_datetime(df_movies['release_date'])

# Creamos una nueva columna con true o false, si la película posee o no página web
df_movies['website'] = df_movies['homepage'].notna()

# Eliminamos la columna homepage
del df_movies['homepage']

# Mantenemos la columna tagline de momento
print('Nulls')
print('----------------------------')
print(df_movies.isnull().sum())

Nulls
----------------------------
budget                    0
genres                    0
keywords                  0
original_language         0
original_title            0
overview                  0
popularity                0
production_companies      0
production_countries      0
release_date              0
revenue                   0
runtime                   0
spoken_languages          0
status                    0
tagline                 840
title                     0
vote_average              0
vote_count                0
cast                      0
crew                      0
website                   0
dtype: int64


In [125]:
#Veamos algunos datos generales de los campos númericos
print(df_movies.describe())

            budget  popularity                   release_date       revenue  \
count      4799.00     4799.00                           4799       4799.00   
mean   29065933.22       21.51  2002-12-25 03:47:26.843092480   82329203.47   
min           0.00        0.00            1916-09-04 00:00:00          0.00   
25%      800000.00        4.69            1999-07-11 12:00:00          0.00   
50%    15000000.00       12.93            2005-09-30 00:00:00   19184015.00   
75%    40000000.00       28.35            2011-02-14 00:00:00   92956519.00   
max   380000000.00      875.58            2017-02-03 00:00:00 2787965087.00   
std    40732511.64       31.82                            NaN  162907644.10   

       runtime  vote_average  vote_count  weighted_average  
count  4799.00       4799.00     4799.00           4799.00  
mean    107.66          6.17      699.43              6.27  
min      14.00          0.50        1.00              4.77  
25%      94.00          5.60       59.00    

In [119]:
#Veamos cuantos campos en cero tenemos
nun_missing = (df_movies[['budget','popularity','revenue','runtime','vote_average','vote_count']] == 0).sum()
print(nun_missing)

budget          1036
popularity         0
revenue         1423
runtime           34
vote_average      61
vote_count        60
dtype: int64


In [120]:
# Solucionemos algunos valores en cero
promedios = df_movies[['runtime','vote_average','vote_count']].mean()
print('Promedios')
print('----------------------')
print(promedios)

#Reemplacemos todos los campos en cero por NaN
df_movies[['runtime','vote_average','vote_count']] = df_movies[['runtime','vote_average','vote_count']].replace(0, np.nan)

df_movies[['runtime','vote_average','vote_count']] = df_movies[['runtime','vote_average','vote_count']].fillna(promedios)

nun_missing = (df_movies[['budget','popularity','revenue','runtime','vote_average','vote_count']] == 0).sum()

print('')
print('Campos con Ceros')
print('------------------------')
print(nun_missing)


Promedios
----------------------
runtime        106.90
vote_average     6.09
vote_count     690.79
dtype: float64

Campos con Ceros
------------------------
budget          1036
popularity         0
revenue         1423
runtime            0
vote_average       0
vote_count         0
dtype: int64


In [121]:
# Vamos a crear una nueva variable weight average o puntuación ponderada para ayudarnos en el análisis
# Si bien tenemos el voto promedio ya como dato (vote_average), no es lo mismo una pelicula con 1 voto y 9 de voto promedio que otra con 500 votos y un voto promedio de 8.5 por eso
# usaremos la siguiente fórmula usada por IMDB para puntuar mejor las películas

v= df_movies['vote_count'] # Cantidad de votos
R=df_movies['vote_average'] # Voto Promedio
C=df_movies['vote_average'].mean()
m=df_movies['vote_count'].quantile(0.7)

df_movies['weighted_average'] = (R*v+C*m)/(v+m)

#Veamos que metricas nos arroja
df_movies['weighted_average'].describe()

count   4799.00
mean       6.27
std        0.38
min        4.77
25%        6.11
50%        6.18
75%        6.36
max        8.34
Name: weighted_average, dtype: float64

In [122]:
print('Nulls')
print('-----------------------------')
print(df_movies.isnull().sum())

Nulls
-----------------------------
budget                    0
genres                    0
keywords                  0
original_language         0
original_title            0
overview                  0
popularity                0
production_companies      0
production_countries      0
release_date              0
revenue                   0
runtime                   0
spoken_languages          0
status                    0
tagline                 840
title                     0
vote_average              0
vote_count                0
cast                      0
crew                      0
website                   0
weighted_average          0
dtype: int64


In [123]:
# Pasemos los campos con datos en formato Json a lista
def convert(texto):
    lista=[]
    for i in ast.literal_eval(texto):
        lista.append(i['name'])
    return lista

df_movies['genres'] = df_movies['genres'].apply(convert)
df_movies['keywords'] = df_movies['keywords'].apply(convert)
df_movies['spoken_languages'] = df_movies['spoken_languages'].apply(convert)

def convert(texto):
    lista=[]
    contador=0
    for i in ast.literal_eval(texto):
        if contador<3:
            lista.append(i['name'])
        contador+=1
    return lista

# Nos quedamos solo con los 3 principales actores de la pelicula
df_movies['cast'] = df_movies['cast'].apply(convert)

def convert(texto):
    lista=[]
    contador=0
    for i in ast.literal_eval(texto):
        if contador<1:
            lista.append(i['name'])
        contador+=1
    return lista

# Filtramos para quedarnos con la principal compañia productora, principal pais y el director
df_movies['production_companies'] = df_movies['production_companies'].apply(convert)
df_movies['production_countries'] = df_movies['production_countries'].apply(convert)
df_movies['crew'] = df_movies['crew'].apply(convert)

df_movies.head()

,budget,genres,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,website,weighted_average
id,,,,,,,,,,,,,,,,,,,,,
19995,237000000,"[Action, Adventure, Fantasy, Science Fiction]","[culture clash, future, space war, space colon...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.44,[Ingenious Film Partners],[United States of America],2009-12-10,...,"[English, Español]",Released,Enter the World of Pandora.,Avatar,7.20,11800.00,"[Sam Worthington, Zoe Saldana, Sigourney Weaver]",[Stephen E. Rivkin],True,7.15
285,300000000,"[Adventure, Fantasy, Action]","[ocean, drug abuse, exotic island, east india ...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.08,[Walt Disney Pictures],[United States of America],2007-05-19,...,[English],Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.90,4500.00,"[Johnny Depp, Orlando Bloom, Keira Knightley]",[Dariusz Wolski],True,6.81
206647,245000000,"[Action, Adventure, Crime]","[spy, based on novel, secret agent, sequel, mi...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.38,[Columbia Pictures],[United Kingdom],2015-10-26,...,"[Français, English, Español, Italiano, Deutsch]",Released,A Plan No One Escapes,Spectre,6.30,4466.00,"[Daniel Craig, Christoph Waltz, Léa Seydoux]",[Thomas Newman],True,6.28
49026,250000000,"[Action, Crime, Drama, Thriller]","[dc comics, crime fighter, terrorist, secret i...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.31,[Legendary Pictures],[United States of America],2012-07-16,...,[English],Released,The Legend Ends,The Dark Knight Rises,7.60,9106.00,"[Christian Bale, Michael Caine, Gary Oldman]",[Hans Zimmer],True,7.51
49529,260000000,"[Action, Adventure, Science Fiction]","[based on novel, mars, medallion, space travel...",en,John Carter,"John Carter is a war-weary, former military ca...",43.93,[Walt Disney Pictures],[United States of America],2012-03-07,...,[English],Released,"Lost in our world, found in another.",John Carter,6.10,2124.00,"[Taylor Kitsch, Lynn Collins, Samantha Morton]",[Andrew Stanton],True,6.12


In [124]:
df_revenue = df_movies[df_movies['revenue'] > 0]
df_revenue.describe()

,budget,popularity,release_date,revenue,runtime,vote_average,vote_count,weighted_average
count,3376.00,3376.00,3376,3376.00,3376.00,3376.00,3376.00,3376.00
mean,38884242.35,28.26,2002-05-23 04:06:58.009478784,117031352.92,110.41,6.31,944.83,6.32
min,0.00,0.02,1916-09-04 00:00:00,5.00,41.00,1.90,1.00,4.78
25%,8500000.00,9.96,1998-10-21 06:00:00,15352895.25,96.00,5.80,161.00,6.08
50%,25000000.00,19.76,2005-09-02 12:00:00,51751835.00,106.00,6.30,441.00,6.21
75%,52000000.00,36.43,2010-12-10 06:00:00,140165095.50,121.00,6.90,1091.25,6.48
max,380000000.00,875.58,2016-09-09 00:00:00,2787965087.00,338.00,10.00,13752.00,8.34
std,44204897.68,35.62,NaN,183483090.42,21.03,0.87,1392.67,0.44


In [126]:
# Supongamos que tienes un DataFrame llamado 'df' con las columnas 'budget' y 'revenue'.

# Define los pesos
peso_presupuesto = 0.7
peso_ganancia = 0.3

# Selecciona una película específica
pelicula_seleccionada = df_revenue.loc[0]  # Cambia el índice según la película que desees calcular

# Extrae el presupuesto y la ganancia de la película seleccionada
presupuesto_pelicula = pelicula_seleccionada['budget']
ganancia_pelicula = pelicula_seleccionada['revenue']

# Calcula la ganancia ponderada
ganancia_ponderada = (peso_presupuesto * presupuesto_pelicula + peso_ganancia * ganancia_pelicula) / (peso_presupuesto + peso_ganancia)

# El resultado es la ganancia ponderada para la película seleccionada
print("Ganancia Ponderada:", ganancia_ponderada)

KeyError: 0